<span style='color:darkcyan'><font size="3">__Juan Sebastian Deslarzes__</font></span><br>
<span style='color:darkcyan'><font size="3">__Chloé De Ancos__</font></span><br>
<span style='color:darkcyan'><font size="3">__Anaïs Burrus__</font></span><br>
<span style='color:darkcyan'><font size="3">__Margot Chapot__</font></span><br>
<span style='color:darkcyan'><font size="3">__Eva Sarlin__</font></span><br>

In [2]:
import matplotlib.pyplot as plt 
%matplotlib inline
import pandas as pd
import numpy as np
from skimage import io
from IPython import display
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
import seaborn as sns
sns.set(color_codes=True)

Loading BokehJS ...

In [3]:
from sklearn.base import BaseEstimator, TransformerMixin
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk import pos_tag
from collections import Counter 
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import classification_report
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import re
import nltk
from sklearn.datasets import load_files
nltk.download('stopwords')
import pickle
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.linear_model import LogisticRegression
import string

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/evasarlin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<span style='color:mediumpurple'><font size="6">__Data Loading__</font></span><br>

In [4]:
sample_submission_url = 'https://raw.githubusercontent.com/esarlin/Challenge-2-CDAW/main/sitc-challenge2-2022/sample_submission.csv'
test_nolabel_url= 'https://raw.githubusercontent.com/esarlin/Challenge-2-CDAW/main/sitc-challenge2-2022/test_nolabel.csv'
train_url = 'https://raw.githubusercontent.com/esarlin/Challenge-2-CDAW/main/sitc-challenge2-2022/train.csv'

sample_submissio<span style='color:mediumpurple'><font size="6">__Data Visualization__</font></span><br>n = pd.read_csv(sample_submission_url)
test_nolabel = pd.read_csv(test_nolabel_url)
train = pd.read_csv(train_url)

<span style='color:mediumpurple'><font size="6">__Data Visualization__</font></span><br>

In [20]:
train.head()

,id,label,statement,subject,speaker,speaker_job,state_info,party_affiliation
0,81f884c64a7,1,China is in the South China Sea and (building)...,"china,foreign-policy,military",donald-trump,President-Elect,New York,republican
1,30c2723a188,0,With the resources it takes to execute just ov...,health-care,chris-dodd,U.S. senator,Connecticut,democrat
2,6936b216e5d,0,The (Wisconsin) governor has proposed tax give...,"corporations,pundits,taxes,abc-news-week",donna-brazile,Political commentator,"Washington, D.C.",democrat
3,b5cd9195738,1,Says her representation of an ex-boyfriend who...,"candidates-biography,children,ethics,families,...",rebecca-bradley,u.s. senator,unknown,none
4,84f8dac7737,0,At protests in Wisconsin against proposed coll...,"health-care,labor,state-budget",republican-party-wisconsin,u.s. senator,Wisconsin,republican


In [21]:
print('Data train dimensions : ', train.shape)
print('Unique values for label : ', train['label'].unique())
print("Number of unique values for statement: ", train['statement'].unique().size)
print("Number of unique values for subject: ", (train['subject'].unique()).size)
print("Number of unique values for speaker: ", (train['speaker'].unique()).size)
print("Number of unique values for speaker job: ", (train['speaker_job'].unique()).size)
print("Number of unique values for state information: ", (train['state_info'].unique()).size)
print("Number of unique values for party affiliation: ", (train['party_affiliation'].unique()).size)

Data train dimensions :  (8950, 8)
Unique values for label :  [1 0]
Number of unique values for statement:  8939
Number of unique values for subject:  3409
Number of unique values for speaker:  2634
Number of unique values for speaker job:  1092
Number of unique values for state information:  80
Number of unique values for party affiliation:  24


<div class="alert alert-block alert-info">
    <span style='color:darkblue'><font size="4"><b>CONCLUSION DATA VISUALIZATION</b></font></span><br><br> <span style='color:black'> As we can see we have a training set of 8950 instances, with 7 columns. We face a classification problem with the target being the column "label" taking as values 0 or 1. We have 6 features, each of them having a large number of possible values. This is due to the fact that these features contains strings, generating a large number of possibilities. The challenge is going to find a way to process these strings in order to use them as input with classification machine learning algorithms.  </span>
</div>

<span style='color:mediumpurple'><font size="6">__Feature Selection and Cleaning__</font></span><br>

In [5]:
train['speaker_job']= train['speaker_job'].fillna('u.s. senator')
test_nolabel['speaker_job']= test_nolabel['speaker_job'].fillna('u.s. senator')

train['state_info']= train['state_info'].fillna('unknown')
test_nolabel['state_info']= test_nolabel['state_info'].fillna('unknown')

In [6]:
cvec=CountVectorizer()

X1 = train['statement']
y = train['label']
X_test1 = test_nolabel['statement']

X2 = train['subject']
X_test2 = test_nolabel['subject']

X3 = train['speaker']
X_test3 = test_nolabel['speaker']

X4 = train['speaker_job']
X_test4 = test_nolabel['speaker_job']

X5 = train['state_info']
X_test5 = test_nolabel['state_info'] 

X6 = train['party_affiliation']
X_test6 = test_nolabel['party_affiliation']

<span style='color:mediumpurple'><font size="6">__NLP__</font></span><br>

<span style='color:steelblue'><font size="3">__The idea is to create dataframes for train and test for each features and to concatenate them at the end with the aim to pass the total dataframe to a classifier. As the classifier will only accept numerical values we need to vectorize the content of each features.__</font></span><br>

In [7]:
cvec=CountVectorizer(stop_words='english').fit(X1)
statement_train=pd.DataFrame(cvec.transform(X1).todense(),columns=cvec.get_feature_names_out())
statement_test=pd.DataFrame(cvec.transform(X_test1).todense(),columns=cvec.get_feature_names_out())

In [8]:
cvec=CountVectorizer(stop_words='english').fit(X2)
subject_train= pd.DataFrame(cvec.transform(X2).todense(),columns=cvec.get_feature_names_out())
subject_test= pd.DataFrame(cvec.transform(X_test2).todense(),columns=cvec.get_feature_names_out())

In [9]:
cvec=CountVectorizer(stop_words='english').fit(X3)
speaker_train= pd.DataFrame(cvec.transform(X3).todense(),columns=cvec.get_feature_names_out())
speaker_test= pd.DataFrame(cvec.transform(X_test3).todense(),columns=cvec.get_feature_names_out())

In [10]:
cvec=CountVectorizer(stop_words='english').fit(X4)
speakerjob_train= pd.DataFrame(cvec.transform(X4).todense(),columns=cvec.get_feature_names_out())
speakerjob_test= pd.DataFrame(cvec.transform(X_test4).todense(),columns=cvec.get_feature_names_out())

In [11]:
cvec=CountVectorizer(stop_words='english').fit(X5)
state_train= pd.DataFrame(cvec.transform(X5).todense(),columns=cvec.get_feature_names_out())
state_test= pd.DataFrame(cvec.transform(X_test5).todense(),columns=cvec.get_feature_names_out())

In [12]:
cvec=CountVectorizer(stop_words='english').fit(X6)
party_train= pd.DataFrame(cvec.transform(X6).todense(),columns=cvec.get_feature_names_out())
party_test= pd.DataFrame(cvec.transform(X_test6).todense(),columns=cvec.get_feature_names_out())

<span style='color:steelblue'><font size="3">__Now we have 6 different pairs of dataframe with the features content vectorized. Let's put them together :__</font></span><br>

In [13]:
total_train=pd.concat([statement_train, subject_train, speaker_train, speakerjob_train, state_train, party_train], axis=1)
total_test=pd.concat([statement_test, subject_test, speaker_test, speakerjob_test, state_test , party_test], axis=1)

In [14]:
print (total_train.shape)
print (total_test.shape)

(8950, 15503)
(3836, 15503)


<span style='color:steelblue'><font size="3">__Now that we have one big dataframe with numerical values we can pass it to a classifier:__</font></span><br>

<span style='color:mediumpurple'><font size="6">__Machine Learning Classification__</font></span><br>

In [16]:
lr=LogisticRegression()

In [17]:
lr.fit(total_train,y)

/Users/evasarlin/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

<div class="alert alert-block alert-info">
    <span style='color:darkblue'><font size="4"><b>CONCLUSION ALGORITHMS</b></font></span><br><br> <span style='color:black'> After testing several algorithms and submit their predictions on kaggle, we found that the regression was the best classifier, giving us an accuracy above 57% whereas Random Frest and SVC were not performing well (51% and 50,5%). </span>
</div>

<span style='color:mediumpurple'><font size="6">__Prediction__</font></span><br>

In [18]:
ypred=lr.predict(total_test)

<span style='color:mediumpurple'><font size="6">__Submission__</font></span><br>

In [19]:
id = test_nolabel['id']
dfsubmission = pd.DataFrame({'id': id, 'label': ypred}, columns=["id", "label"])
dfsubmission.to_csv('concatenationSVC.csv', index=False, header=True)

<div class="alert alert-block alert-info">
    <span style='color:purple'><font size="4"><b>CONCLUSION GENERALE</b></font></span><br><br> <span style='color:black'> ici on écrit les conclusions, on peut même écrire en italique <i>voili </i>voilou. </span>
</div>